In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.models import load_model
import pickle
import requests
import pandas as pd
import json

headers = {'X-Mashape-Key': api_key}

### We will be speaking to the Hearthstone API hosted by Mashape. It is free and let's us specify what card information we want to receive via paramaters.

In [6]:
api_key = "hh8DW29tdDmshbghN71NJPI0jgFzp1Ay5MzjsniIz9a8Piq7xp"
def get_cards_by_type(card_type, collectible=1, cost=None, durability=None, health=None, key=api_key):
    endpoint_by_type = f'https://omgvamp-hearthstone-v1.p.mashape.com/cards/types/{card_type}'
    payload = {'collectible': collectible, 'cost': cost, 'durability': durability, 'health': health}
    r = requests.get(endpoint_by_type, params=payload, headers=headers)
    return r.json()

In [10]:
get_cards_by_type('Spell', cost=8)[:2]

[{'artist': 'Andrew Hou',
  'cardId': 'UNG_004',
  'cardSet': "Journey to Un'Goro",
  'collectible': True,
  'cost': 8,
  'dbfId': '41130',
  'faction': 'Neutral',
  'flavor': 'Comes with fries and a drink.',
  'img': 'http://media.services.zam.com/v1/media/byName/hs/cards/enus/UNG_004.png',
  'imgGold': 'http://media.services.zam.com/v1/media/byName/hs/cards/enus/animated/UNG_004_premium.gif',
  'locale': 'enUS',
  'name': 'Dinosize',
  'playerClass': 'Paladin',
  'rarity': 'Epic',
  'text': "Set a minion's Attack and Health to 10.",
  'type': 'Spell'},
 {'artist': 'Anton Magdalina',
  'cardId': 'UNG_854',
  'cardSet': "Journey to Un'Goro",
  'collectible': True,
  'cost': 8,
  'dbfId': '42009',
  'faction': 'Neutral',
  'flavor': 'Bingo! Minion DNA!',
  'img': 'http://media.services.zam.com/v1/media/byName/hs/cards/enus/UNG_854.png',
  'imgGold': 'http://media.services.zam.com/v1/media/byName/hs/cards/enus/animated/UNG_854_premium.gif',
  'locale': 'enUS',
  'mechanics': [{'name': 'D

In [11]:
spells = get_cards_by_type('Spell')
weapons = get_cards_by_type('Weapon')
minions = get_cards_by_type('Minion')

### I'm a fan of saving data offline in case I can't access the API / the API is down / my rates are exceeded.

In [12]:
for cardtype, collection in zip(['minions', 'spells', 'weapons'], [minions, spells, weapons]):
    with open(f'{cardtype}.json', 'w') as outfile:
        json.dump(collection, outfile)

### Now we can load the data offline whenever we want.

In [13]:
with open('data/minions.json') as json_data:
    minions = json.load(json_data)
    
with open('data/spells.json') as json_data:
    spells = json.load(json_data)
    
with open('data/weapons.json') as json_data:
    weapons = json.load(json_data)

### Let's now separate the titles, flavors, texts, mechanics, costs and stats.
#### Each card type has different attributes and design logic behind them, so we want to make educated splits. Moreover, not all fields are of interest for us for now. 

In [15]:
spell_attributes = {"name", "cardId", "cost", "img", "playerClass", "rarity", "text", "flavor", "mechanics"}

minion_attributes = {"name", "cardId", "cost", "health", "attack", "img", "playerClass", "rarity", "text", "flavor", "mechanics"}

weapon_attributes = {"name", "cardId", "cost", "durability", "attack", "img", "playerClass", "rarity", "text", "flavor", "mechanics"}

### For this first generation example we shall funnel all the cards to the above attributes to normalize the data.

In [17]:
def normalize_card(card, attrs):
    
    concise = {a: card[a] if a in card else None for a in attrs}
    
    return concise

spells_concise = [normalize_card(spell_card, spell_attributes) for spell_card in spells]
minions_concise = [normalize_card(minion_card, minion_attributes) for minion_card in minions]
weapons_concise = [normalize_card(weapon_card, weapon_attributes) for weapon_card in weapons]

### The data is now quite ready to be modelled! We can now access the needed fields directly from the correpsonding cardtype set.

## A small LSTM

### Let's get all the card texts.

In [18]:
def get_all_cardtexts():
    
    weapon_texts = [card['text'] for card in weapons if 'text' in card]
    minion_texts = [card['text'] for card in minions if 'text' in card]
    spell_texts = [card['text'] for card in spells if 'text' in card] 
    
    return weapon_texts + minion_texts + spell_texts

### Our average sequence length is ... characters. This is handy to know for the LSTM sequence length parameter. Let's round it down for sake of memorability.

In [21]:
# average_sequence_length = np.mean([len(list(text)) for text in all_cardtexts])
# average_sequence_length
# SEQUENCE_LENGTH = 14

### Our generative model is a character based one, so our input data is a huge list of characters.

In [22]:
# all_text_chars = [ch for one_sentence in all_cardtexts for ch in list(one_sentence)]
all_text_unique_chars = list(set(all_text_chars))
all_text_unique_chars.sort() # THIS IS VERY IMPORTANT TO GET REPRODUCIBLE RESULTS WHEN SAVING THE MODEL, 
                             # OTHERWISE THE CHARACTER MAPPING WILL BE DIFFERENT EACH TIME!

In [23]:
# print(f'Data length: {len(all_text_chars)} characters')
# print(f'Vocabulary size: {len(all_text_unique_chars)} characters')

### The unique characters are the features for our model. Let's numerify them to make them ML ready.

In [24]:
# ix_to_char = {ix:char for ix, char in enumerate(all_text_unique_chars)}
# char_to_ix = {char:ix for ix, char in enumerate(all_text_unique_chars)}

### LSTM expects input of the shape (batch_size, length_of_sequence, number_features)
#### batch_size: amount of sequences which are fed into the network at a single weight update iteration, just as in a regular feedforward neural network
#### length_of_sequence: the amount of "neural networks", the memory or the amount of steps the network looks at at each step. In our example, we want to predict a character given 57 previous characters.
#### number_features: the length of one featurized element. In the case of images it could be padded standardized vectors of pixels. In case of text it is the length of our vocab, because our input is going to be represented by every char in our vocabulary.


In [25]:
# NUMBER_FEATURES = len(all_text_unique_chars)

### Now we will initialize our X and y for the network by creating a correctly shaped zeros array which we will fill iteratively.

In [26]:
# X = np.zeros((int(len(all_text_chars)/SEQUENCE_LENGTH), SEQUENCE_LENGTH, NUMBER_FEATURES))
# y = np.zeros((int(len(all_text_chars)/SEQUENCE_LENGTH), SEQUENCE_LENGTH, NUMBER_FEATURES))

### The idea is that we want to predict the same length sequence as is our input, BUT just shift it with one character. This is common while using LSTM's for prediction as it makes shaping the data convenient.
### Then we practically are predicting one character at a time. For a sequence [c, a, r, r, o] we want to have the sequence [a, r, r, o, t] as output. This way, each char in the input has the following char as label (c -> a, a -> r, r -> r, etc.) 

In [27]:
# for i in range(int(len(all_text_chars)/SEQUENCE_LENGTH)):
#     X_sequence = all_text_chars[i*SEQUENCE_LENGTH:(i+1)*SEQUENCE_LENGTH]  #Get next sequence of length 57 as input.
#     X_sequence_ix = [char_to_ix[value] for value in X_sequence]  # Convert the above sequence to the integer mapping.
#     # TODO: make this one hot encoding differently: Keras or sklearn or something.
#     input_sequence = np.zeros((SEQUENCE_LENGTH, NUMBER_FEATURES))  # Create a skeleton for the input sequence: we create a 2d numpy matrix which has a feature array of 94 
#                                                                    # long for each of the 57 characters in sequence. This way we basically one hot encode our sequences. 
#     for j in range(SEQUENCE_LENGTH):  # The one hot encoding process: we replace a zero with a one on a position in the input sequence which corresponds with the index of a character in our converted array!
#         input_sequence[j][X_sequence_ix[j]] = 1.
#     X[i] = input_sequence  #For each spot in X (which stands for each sequence) we fill in our one hot encoded array!
    
#     #Same for y!
#     y_sequence = all_text_chars[i*SEQUENCE_LENGTH+1:(i+1)*SEQUENCE_LENGTH+1]
#     y_sequence_ix = [char_to_ix[value] for value in y_sequence]
#     target_sequence = np.zeros((SEQUENCE_LENGTH, NUMBER_FEATURES))
#     for j in range(SEQUENCE_LENGTH):
#         target_sequence[j][y_sequence_ix[j]] = 1.
#     y[i] = target_sequence

### Define model hyperparameters: 3 LSTM layers with each having 500 hidden units

In [28]:
HIDDEN_DIM = 500
LAYER_NUM = 3

### Define model architecture (3 LSTM layers with Dropout regularization), a TimeDistributed layer which makes sure all of our LSTM cell states are passed through the Dense layer and not just the last one because the goal is to predict a sequence and not just the last character. Otherwise the loss function will only be calculated for the last input and not the whole sequence.
### Because we are essentially predicting classes (characters are categorical) we use categorical crossentropy as our loss function and rmsprop proved to be a good optimizer for RNN tasks.

In [29]:
model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, NUMBER_FEATURES), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
    model.add(Dropout(0.25))
model.add(TimeDistributed(Dense(NUMBER_FEATURES)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [30]:
def generate_text(model, length, ixtochars):
    
    hele_tekst = []
    
    ix = [np.random.randint(NUMBER_FEATURES)]  # We start with a random character.
    y_char = [ixtochars[ix[0]]]  # Get the random char's string variant.
    
    X = np.zeros((1, length, NUMBER_FEATURES)) # Generate a placeholder empty numpy array which contains an zeros array for each of the amount of letters 
                                                # that we want to generate and each of these arrays is as long as is our vocabulary, just as in the data 
                                                # prep step.
    for i in range(length):
        X[0, i, :][ix[0]] = 1
        print(ixtochars[ix[0]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1) # Given a single character one-hot encoding, generate predictions for the next one 
                                                            # and choose the best one.
        hele_tekst.append(ixtochars[ix[0]])  # Append the predicted character to the list, repeat it for length steps.
    return hele_tekst

### We specify the batch size (just the same as in a simple Feedforward ANN) and the length of card text we want to generate. 
### In an endless loop, we fit the model for one epoch, so we let the network go through the whole data with batch size 50, generate the text to just see intermediate steps of the network and then fit again on the same model but now with different batch. 
### We also want to save weights of the model every 200 epochs to be able to compare the performance / load weights into the same architecture later!

In [31]:
nb_epoch = 0
BATCH_SIZE = 50
GENERATE_LENGTH = 57
while True:
    print('\n')
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, nb_epoch=1)
    nb_epoch += 1
    generate_text(model, GENERATE_LENGTH, ix_to_char)
    if nb_epoch % 200 == 0:
        model.save_weights('nametext_cps/checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, nb_epoch))